# ETL da camada Silver para camada Gold - Microsoft Security Incident Prediction

Este notebook realiza o ETL (Extract, Transform, Load) dos dados da camada Silver para a camada Gold. 
Focamos em manter apenas colunas relevantes para construção de dashboards futuros, como agregações temporais, geográficas e por severidade de incidentes.

## Colunas Mantidas para Dashboard
- timestamp: Para tendências temporais.
- orgid: Agregação por organização.
- detectorid: Detetores de alertas.
- alerttitle: Títulos de alertas.
- category: Categorias de incidentes.
- mitretechniques: Técnicas MITRE.
- incidentgrade: Severidade (target principal).
- entitytype: Tipos de entidades.
- evidencerole: Papel da evidência.
- osfamily: Família de SO.
- osversion: Versão de SO.
- lastverdict: Veredito final.
- countrycode, state, city: Localização geográfica.


## EXTRACT

Extraímos os dados do arquivo CSV da camada Silver.


In [1]:
import pandas as pd
import numpy as np
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')

# Configuração do caminho dos dados
data_layer_filepath = '../../'

# Carregamento do dataset Silver
silver_file = data_layer_filepath + 'data_layer/silver/security_incident_prediction_silver.csv'
df = pd.read_csv(silver_file, low_memory=False)
print("Dataset Silver carregado com sucesso!")
print(f"Dimensões do dataset: {df.shape}")
df.head()


Dataset Silver carregado com sucesso!
Dimensões do dataset: (6850703, 37)


,id,orgid,incidentid,alertid,timestamp,detectorid,alerttitle,category,mitretechniques,incidentgrade,...,oauthapplicationid,filename,folderpath,resourceidname,osfamily,osversion,lastverdict,countrycode,state,city
0,180388628218,0,612,123247,94972,7,6,10,1183,2,...,881,289573,117668,3586,5,66,5,31,6,3
1,455266534868,88,326,210035,668070,58,43,7,1183,1,...,881,289573,117668,3586,5,66,5,242,1445,10630
2,1056561957389,809,58352,712507,615220,423,298,10,859,1,...,881,289573,117668,3586,5,66,4,242,1445,10630
3,1279900258736,92,32992,774301,470878,2,2,1,1183,0,...,881,289573,117668,3586,5,66,4,242,1445,10630
4,214748368522,148,4359,188041,707245,9,74,6,1183,2,...,881,289573,117668,3586,5,66,5,242,1445,10630


## TRANSFORM

Realizamos transformações: seleção de colunas relevantes, conversão de tipos e criação de features derivadas para dashboards.


### Seleção de Colunas Relevantes para Dashboard

Mantemos apenas colunas úteis para visualizações e agregações em dashboards.


In [2]:
# Colunas relevantes para dashboard
relevant_columns = [
    'timestamp', 'orgid', 'detectorid', 'alerttitle',
    'category', 'mitretechniques', 'incidentgrade',
    'entitytype', 'evidencerole', 'osfamily', 'osversion',
    'lastverdict', 'countrycode', 'state', 'city'
]

# Verificar se todas as colunas existem
missing_cols = [col for col in relevant_columns if col not in df.columns]
if missing_cols:
    print(f" Colunas ausentes no Silver: {missing_cols}")
else:
    print("Todas as colunas relevantes encontradas.")

# Selecionar colunas
df_gold = df[relevant_columns].copy()
print(f"Dimensões após seleção: {df_gold.shape}")
print(f"Colunas mantidas: {len(relevant_columns)}")


Todas as colunas relevantes encontradas.
Dimensões após seleção: (6850703, 15)
Colunas mantidas: 15


### Conversão de Tipos e Features Derivadas

Convertemos timestamp para datetime e criamos colunas derivadas (ano, mês, dia) para agregações temporais em dashboards.


In [3]:
# Converter timestamp para datetime (se já encoded, ajuste conforme necessário)
if df_gold['timestamp'].dtype == 'object':
    df_gold['timestamp'] = pd.to_datetime(df_gold['timestamp'], errors='coerce')
else:
    # Se já encoded numericamente, assuma que é um label; para dashboard, talvez reverta ou ignore
    print("Timestamp já encoded numericamente. Mantendo como está para agregações.")

# Criar features derivadas temporais (se timestamp for datetime)
if pd.api.types.is_datetime64_any_dtype(df_gold['timestamp']):
    df_gold['year'] = df_gold['timestamp'].dt.year
    df_gold['month'] = df_gold['timestamp'].dt.month
    df_gold['day'] = df_gold['timestamp'].dt.day
    df_gold['hour'] = df_gold['timestamp'].dt.hour
    print("Features temporais derivadas criadas: year, month, day, hour")
else:
    print("Não foi possível criar features temporais (timestamp não é datetime).")

# Verificar tipos de dados
print("\nTipos de dados após transformações:")
print(df_gold.dtypes)


Timestamp já encoded numericamente. Mantendo como está para agregações.
Não foi possível criar features temporais (timestamp não é datetime).

Tipos de dados após transformações:
timestamp          int64
orgid              int64
detectorid         int64
alerttitle         int64
category           int64
mitretechniques    int64
incidentgrade      int64
entitytype         int64
evidencerole       int64
osfamily           int64
osversion          int64
lastverdict        int64
countrycode        int64
state              int64
city               int64
dtype: object


### Tratamento de Duplicatas e Qualidade Final

Removemos duplicatas e verificamos qualidade.


In [4]:
# Remover duplicatas
duplicates_before = df_gold.duplicated().sum()
df_gold = df_gold.drop_duplicates()
duplicates_after = df_gold.duplicated().sum()
print(f"Duplicatas removidas: {duplicates_before - duplicates_after}")

# Verificar valores ausentes (deve ser zero da Silver)
missing_final = df_gold.isnull().sum().sum()
print(f"Total de valores ausentes: {missing_final}")
if missing_final > 0:
    print(" Valores ausentes encontrados! Tratando com mediana/moda.")
    for col in df_gold.columns:
        if df_gold[col].isnull().any():
            if df_gold[col].dtype in ['int64', 'float64']:
                df_gold[col].fillna(df_gold[col].median(), inplace=True)
            else:
                df_gold[col].fillna(df_gold[col].mode()[0], inplace=True)


Duplicatas removidas: 3224787
Total de valores ausentes: 0


## LOAD

Carregamos os dados processados para a camada Gold.


### Salvando os Dados Processados em CSV

Salvamos os dados na camada Gold.


In [5]:
# Criar diretório gold se não existir
gold_dir = data_layer_filepath + 'data_layer/gold/'
if not os.path.exists(gold_dir):
    os.makedirs(gold_dir)
    print(f"Diretório criado: {gold_dir}")

# Salvar dados processados
output_file = gold_dir + 'security_incident_prediction_gold.csv'
df_gold.to_csv(output_file, index=False)

print(f"Dataset da camada Gold salvo com sucesso!")
print(f"Arquivo: {output_file}")
print(f"Dimensões: {df_gold.shape}")
print(f"Tamanho do arquivo: {os.path.getsize(output_file) / 1024**2:.2f} MB")


Diretório criado: ../../data_layer/gold/
Dataset da camada Gold salvo com sucesso!
Arquivo: ../../data_layer/gold/security_incident_prediction_gold.csv
Dimensões: (3625916, 15)
Tamanho do arquivo: 179.36 MB


### Verificação Final da Qualidade dos Dados

Verificação final.


In [6]:
print("=== VERIFICAÇÃO FINAL DA QUALIDADE DOS DADOS (GOLD) ===")
print(f"Dimensões finais: {df_gold.shape}")
print(f"Memória utilizada: {df_gold.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Total de valores ausentes: {df_gold.isnull().sum().sum()}")
print(f"\nTipos de dados finais:")
print(df_gold.dtypes.value_counts())
print("\n Verificação de qualidade concluída! Dados prontos para dashboard.")


=== VERIFICAÇÃO FINAL DA QUALIDADE DOS DADOS (GOLD) ===
Dimensões finais: (3625916, 15)
Memória utilizada: 442.62 MB
Total de valores ausentes: 0

Tipos de dados finais:
int64    15
Name: count, dtype: int64

 Verificação de qualidade concluída! Dados prontos para dashboard.
